In [95]:
import xml.etree.ElementTree as ET
import datetime
import os
import requests
import json
from requests_kerberos import HTTPKerberosAuth
import time

# !grep  -A1 fs.azure.ext.cab.address /etc/hadoop/conf/core-site.xml  | tail -1

def get_token(id_broker_group = "eur-app-aiu-dev", dl_name = "dev"):
    id_broker=f'https://dl-{dl_name}-idbroker0.az-{dl_name}.x9er-zkvz.cloudera.site:8444/gateway/'
    r = requests.get(f"{id_broker}dt/knoxtoken/api/v1/token", auth=HTTPKerberosAuth())
    token=r.json()['access_token']

    url = f"{id_broker}azure-cab/cab/api/v1/credentials/group/{id_broker_group}"
    headers = {
            'Authorization': "Bearer "+ token,
            'cache-control': "no-cache",
            'Accept': 'application/json' 
    }
    token = requests.request("GET", url, headers=headers).json()
    return token

def get_adls_gen2_folder_contents(storage_account, container, folder, access_token):
    """
    List the contents of a folder in an Azure Data Lake Storage Gen2 filesystem.
    
    Args:
        storage_account (str): The storage account name.
        container (str): The container name within the ADLS Gen2 account.
        folder (str): The folder path to list contents of.
        access_token (str): The OAuth2 access token for authentication.
    
    Returns:
        response.content: The HTTP response content from the Azure Storage REST API.
    """
    date = datetime.datetime.utcnow().strftime('%a, %d %b %Y %H:%M:%S GMT')
    url = f'https://{storage_account}.dfs.core.windows.net/{container}?resource=filesystem&recursive=false&directory={folder}'
    headers = {
        'Authorization': "Bearer " + access_token,
        'x-ms-version': '2019-12-12',
        'x-ms-date': date,
    }
    response = requests.get(url, headers=headers)
    return response.json()

# Example usage
storage_account = "cdpdldev0"  # Adjusted to the account mentioned in the HDFS command
container = "storage-fs"  # Kept as is, per your initial input
folder = 'data/project/aiu.other/'  # Adjusted folder path based on the HDFS command output
access_token = get_token()['access_token']  # Replace YOUR_ACCESS_TOKEN with your actual access token

contents = get_adls_gen2_folder_contents(storage_account, container, folder, access_token)
print(contents)

{'paths': [{'contentLength': '0', 'etag': '0x8DC42A3CD447BF1', 'group': '$superuser', 'isDirectory': 'true', 'lastModified': 'Tue, 12 Mar 2024 14:50:47 GMT', 'name': 'data/project/aiu.other/ec-datadump', 'owner': '739339c7-bf30-4d17-9760-77cdcc55f87c', 'permissions': 'rwxrwx--x+'}, {'contentLength': '0', 'etag': '0x8DC42A3BA7366A1', 'group': '$superuser', 'isDirectory': 'true', 'lastModified': 'Tue, 12 Mar 2024 14:50:15 GMT', 'name': 'data/project/aiu.other/processed_data', 'owner': '739339c7-bf30-4d17-9760-77cdcc55f87c', 'permissions': 'rwxrwx--x+'}]}
